## Step 1: link to the API, pull data

In [14]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import requests
from config import key
from config import gkey
import time
import datetime
import pymongo

In [ ]:
#get state code number --CA 06
state_url = "https://aqs.epa.gov/data/api/list/states?email="+gkey+"&key="+key
response = requests.get(state_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
state_df = pd.DataFrame.from_dict(response_json_data)
state_df.head()

In [ ]:
#get SF code -- 075
city_url = "https://aqs.epa.gov/data/api/list/countiesByState?email="+gkey +"&key="+key+"&state=06"
print(city_url)
response = requests.get(city_url)
response_json = response.json()
response_json_data = response_json['Data']
#print(response_json_data)
city_df = pd.DataFrame.from_dict(response_json_data)
city_df.head()

In [ ]:
#CBSAs code": "41860",
#      "value_represented": "San Francisco-Oakland-Hayward, CA

#  "Parameter code": "PM2.5 CONT NONREF",
#       "value_represented": "PM2.5 Continuous, Nonreference Methods"
#     },
#     {
#       "code": "PM2.5 MASS/QA",
#       "value_represented": "PM2.5 Mass and QA Parameters"


# "CRITERIA code": "88101",
#       "value_represented": "PM2.5 - Local Conditions"

In [ ]:
#get current 5y PM 2.5 data for SF
#https://aqs.epa.gov/data/api/sampleData/byCBSA?email=test@aqs.api&key=test&param=42602&bdate=20170101&edate=20170101&cbsa=16740
#sf_url = "https://aqs.epa.gov/data/api/sampleData/byCBSA?email="+gkey+"&key="+key+"&param=88101&bdate=20190101&edate=20190101&cbsa=41860"

sf_url = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county=075"
print(sf_url)
response = requests.get(sf_url)
response_json = response.json()
#print(response_json)
response_json_data = response_json['Data']
# #print(response_json_data)
sf_df = pd.DataFrame.from_dict(response_json_data)
sf_df.head()
#https://aqs.epa.gov/data/api/monitors/byState?email=test@aqs.api&key=test&param=42401&bdate=20150501&edate=20150502&state=15

In [ ]:
# get 5year California data by county
county_codes = city_df['code'].tolist()
county_codes_10 = county_codes[0:10]
county_codes_10

In [ ]:
total_county_data_10 = pd.DataFrame()
for i in county_codes_10:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_10 = total_county_data_10.append(county_data)
    time.sleep(2)
total_county_data_10.head()

In [ ]:
total_county_data_10.to_csv("ca_10.csv")

In [ ]:
county_codes = city_df['code'].tolist()
county_codes_40 = county_codes[31:40]
county_codes_40

In [ ]:
total_county_data_40 = pd.DataFrame()
for i in county_codes_40:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_40 = total_county_data_40.append(county_data)
    time.sleep(2)
total_county_data_40.head()

In [ ]:
total_county_data_40.to_csv("ca_40.csv")

In [ ]:
county_codes = city_df['code'].tolist()
county_codes_30 = county_codes[21:30]
county_codes_30

In [ ]:
total_county_data_30 = pd.DataFrame()
for i in county_codes_30:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_30 = total_county_data_30.append(county_data)
    time.sleep(2)
total_county_data_30.head()

In [ ]:
total_county_data_30.to_csv("ca_30.csv")

In [ ]:
county_codes = city_df['code'].tolist()
county_codes_50 = county_codes[41:50]
county_codes_50

In [ ]:
total_county_data_50 = pd.DataFrame()
for i in county_codes_50:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_50 = total_county_data_50.append(county_data)
    time.sleep(2)
total_county_data_50.head()

In [ ]:
total_county_data_50.to_csv("ca_50.csv")

In [ ]:
county_codes = city_df['code'].tolist()
county_codes_60 = county_codes[51:58]
county_codes_60

In [ ]:
total_county_data_60 = pd.DataFrame()
for i in county_codes_60:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_60 = total_county_data_60.append(county_data)
    time.sleep(2)
total_county_data_60.head()

In [ ]:
total_county_data_60.to_csv("ca_60.csv")

In [ ]:
county_codes = city_df['code'].tolist()
county_codes_20 = county_codes[11:20]
county_codes_20

In [ ]:
total_county_data_20 = pd.DataFrame()
for i in county_codes_20:
    county_data = "https://aqs.epa.gov/data/api/dailyData/byCounty?email="+gkey+"&key="+key+"&param=88101&bdate=20140301&edate=20190331&state=06&county="+i
    print(county_data)
    response = requests.get(county_data)
    response_json = response.json()
    response_json_data = response_json['Data']
    county_data =pd.DataFrame.from_dict(response_json_data)
    total_county_data_20 = total_county_data_20.append(county_data)
    time.sleep(2)
total_county_data_20.head()

In [ ]:
total_county_data_20.to_csv("ca_20.csv")

## Step 2: Clean data with pandas

In [5]:
total_county_data_10 = df = pd.read_csv('Resources/ca_10.csv')
total_county_data_20 = df = pd.read_csv('Resources/ca_20.csv')
total_county_data_30 = df = pd.read_csv('Resources/ca_30.csv')
total_county_data_40 = df = pd.read_csv('Resources/ca_40.csv')
total_county_data_50 = df = pd.read_csv('Resources/ca_50.csv')
total_county_data_60 = df = pd.read_csv('Resources/ca_60.csv')

In [6]:
frames = [total_county_data_10, total_county_data_20, total_county_data_30,total_county_data_40,total_county_data_50,total_county_data_60]

In [7]:
combined_ca = pd.concat(frames)
combined_ca.head()

,Unnamed: 0,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,...,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,0,18.0,4.2,"San Francisco-Oakland-Hayward, CA",41860.0,Livermore,Alameda,1,2019-03-31,2019-08-27,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,7,California,6,Micrograms/cubic meter (LC),Y
1,1,16.0,3.9,"San Francisco-Oakland-Hayward, CA",41860.0,Livermore,Alameda,1,2019-03-30,2019-08-27,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,7,California,6,Micrograms/cubic meter (LC),Y
2,2,8.0,1.8,"San Francisco-Oakland-Hayward, CA",41860.0,Livermore,Alameda,1,2019-03-29,2019-08-27,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,7,California,6,Micrograms/cubic meter (LC),Y
3,3,19.0,4.5,"San Francisco-Oakland-Hayward, CA",41860.0,Livermore,Alameda,1,2019-03-28,2019-08-27,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,7,California,6,Micrograms/cubic meter (LC),Y
4,4,9.0,2.1,"San Francisco-Oakland-Hayward, CA",41860.0,Livermore,Alameda,1,2019-03-27,2019-08-27,...,88101,3,PM25 24-hour 2006,24-HR BLK AVG,793 Rincon Ave.,7,California,6,Micrograms/cubic meter (LC),Y


In [6]:
combined_ca.count()

Unnamed: 0             780214
aqi                    651335
arithmetic_mean        780214
cbsa                   747466
cbsa_code              747466
city                   780214
county                 780214
county_code            780214
date_local             780214
date_of_last_change    780214
datum                  780214
event_type             780214
first_max_hour         780214
first_max_value        780214
latitude               780214
local_site_name        779904
longitude              780214
method                 780214
method_code            780214
observation_count      780214
observation_percent    780214
parameter              780214
parameter_code         780214
poc                    780214
pollutant_standard     651335
sample_duration        780214
site_address           780214
site_number            780214
state                  780214
state_code             780214
units_of_measure       780214
validity_indicator     780214
dtype: int64

In [8]:
ca_clean = combined_ca[['aqi','cbsa','city','county','county_code','date_local','parameter_code','sample_duration','pollutant_standard','latitude','longitude','state','state_code']]
ca_clean.head()

,aqi,cbsa,city,county,county_code,date_local,parameter_code,sample_duration,pollutant_standard,latitude,longitude,state,state_code
0,18.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,1,2019-03-31,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,6
1,16.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,1,2019-03-30,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,6
2,8.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,1,2019-03-29,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,6
3,19.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,1,2019-03-28,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,6
4,9.0,"San Francisco-Oakland-Hayward, CA",Livermore,Alameda,1,2019-03-27,88101,24-HR BLK AVG,PM25 24-hour 2006,37.687526,-121.784217,California,6


In [8]:
ca_clean = ca_clean.loc[ca_clean['pollutant_standard'] == "PM25 24-hour 2012"]
ca_clean =ca_clean.loc[ca_clean['sample_duration'] != "1 HOUR"]
ca_clean.count()

aqi                   162822
cbsa                  156150
city                  162822
county                162822
county_code           162822
date_local            162822
parameter_code        162822
sample_duration       162822
pollutant_standard    162822
latitude              162822
longitude             162822
state                 162822
state_code            162822
dtype: int64

In [8]:
# ca_clean.to_excel("ca_clean.xlsx")

In [ ]:
# group data by day and county to get avg aqi
ca_avg = ca_clean.groupby(['date_local','county_code']).mean()
ca_avg = ca_avg.reset_index()
ca_avg['month'] = pd.DatetimeIndex(ca_avg['date_local']).month
ca_avg

In [26]:
ca_avg_month = ca_clean
ca_avg_month['month'] = pd.DatetimeIndex(ca_clean['date_local']).month
ca_avg_month = ca_avg_month.groupby(['month','county_code']).mean()
ca_avg_month = ca_avg_month.reset_index()
ca_avg_month

C:\Users\Kanika\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,month,county_code,aqi,parameter_code,latitude,longitude,state_code
0,1,1,43.203601,88101,37.769031,-122.134485,6
1,1,7,45.022059,88101,39.761680,-121.840470,6
2,1,9,41.676056,88101,38.201850,-120.680277,6
3,1,11,22.090909,88101,39.039648,-122.250764,6
4,1,13,44.747541,88101,37.948039,-122.189212,6
5,1,15,37.967742,88101,41.756130,-124.203470,6
6,1,19,59.907684,88101,36.791732,-119.839939,6
7,1,23,31.168000,88101,40.847867,-124.019147,6
8,1,25,47.921824,88101,32.737699,-115.502965,6
9,1,27,13.066265,88101,36.844137,-118.058711,6


In [10]:
#choose only "pm2.5 24h standard"
sf_df_new = sf_df.loc[sf_df['pollutant_standard']=="PM25 24-hour 2012"]
sf_df_new.head()

NameError: name 'sf_df' is not defined

In [ ]:
#get min,max,mean
aqi_min = sf_df_new['aqi'].min()
aqi_max =sf_df_new['aqi'].max()
aqi_mean = sf_df_new['aqi'].mean()

print(aqi_min)
print(aqi_max)
print(aqi_mean)

In [35]:
# summurize data in categories
size_bins = [0,50,100,150,200,300,500]
size_names= ['good','Moderate','Unhealthy for sensitive groups','unhealthy','very unhealthy','hazardous']
sf_df_bin = sf_df_new[['date_local']].groupby(pd.cut(sf_df_new['aqi'],bins=size_bins,labels=size_names)).count()
sf_df_bin

NameError: name 'sf_df_new' is not defined

## Step 3: export data to the database

In [27]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# 'testdb' is the name of the database
db = client.ca_data

# testcollection is the name of the collection
# collection = db['ca_data']
# collection2 = db['ca_avg']
collection3 = db['ca_avg_month']

# this line stays the same
# collection.insert_many(ca_clean.to_dict('records'))
# collection2.insert_many(ca_avg.to_dict('records'))
collection3.insert_many(ca_avg_month.to_dict('records'))

In [13]:
# query mongodb to get aqi for a specific day
my_collection = db['ca_avg']
mydoc = my_collection.find({'date_local':'2018-11-17'}, {'county_code': 1, 'aqi': 1, '_id': 0})
return_list = []
for x in mydoc:
    if x["aqi"] < 50:
        x["level"] = "good"
    elif x["aqi"] < 100:
        x["level"] = "moderate"
    elif x["aqi"] < 150:
        x["level"] = "unhealthy for sensitive groups"
    elif x["aqi"] < 200:
        x["level"] = "unhealthy"
    elif x["aqi"] < 300:
        x["level"] = "very unhealthy"
    elif x["aqi"] >= 300:
        x["level"] = "hazardous"
    return_list.append(x)
return_list

[{'county_code': 1, 'aqi': 182.83333333333334, 'level': 'unhealthy'},
 {'county_code': 7, 'aqi': 243.0, 'level': 'very unhealthy'},
 {'county_code': 9, 'aqi': 92.0, 'level': 'moderate'},
 {'county_code': 11, 'aqi': 158.0, 'level': 'unhealthy'},
 {'county_code': 13, 'aqi': 197.0, 'level': 'unhealthy'},
 {'county_code': 19, 'aqi': 147.6, 'level': 'unhealthy for sensitive groups'},
 {'county_code': 25, 'aqi': 124.0, 'level': 'unhealthy for sensitive groups'},
 {'county_code': 27, 'aqi': 27.5, 'level': 'good'},
 {'county_code': 29, 'aqi': 53.333333333333336, 'level': 'moderate'},
 {'county_code': 31, 'aqi': 164.5, 'level': 'unhealthy'},
 {'county_code': 37, 'aqi': 64.4, 'level': 'moderate'},
 {'county_code': 39, 'aqi': 156.0, 'level': 'unhealthy'},
 {'county_code': 45, 'aqi': 144.0, 'level': 'unhealthy for sensitive groups'},
 {'county_code': 47, 'aqi': 165.0, 'level': 'unhealthy'},
 {'county_code': 51, 'aqi': 53.0, 'level': 'moderate'},
 {'county_code': 53,
  'aqi': 124.66666666666667,
  

In [36]:
size_bins = [0,50,100,150,200,300,500]
size_names= ['good','Moderate','Unhealthy for sensitive groups','unhealthy','very unhealthy','hazardous']